In [3]:
# %%writefile factor_101.py

import sys
import os 


import QUANTAXIS as QA
from QUANTAXIS.QAAnalysis.QAAnalysis_finance import QAAnalysis_finance
from QUANTAXIS.QASetting.QALocalize import analysis_path
from factorBase import FactorBase
from factorClickhouse import FactorCKClient

from qaenv import (clickhouse_ip, clickhouse_password, clickhouse_port,
                   clickhouse_user)


from ind.Alpha101 import *

from tools import Sample_Tools as smpl
from tools import QAAdapter as qaadapter
# from tools import QAAdapter as qaadapter
# from tools import QA_adapter_get_code_from_block


class alpha101_1(FactorBase):

    # def finit(self):

    #   self.clientr = FactorCKClient(clickhouse_ip, clickhouse_port, user=clickhouse_user, password=clickhouse_password)
    #   self.factor_name = 'alpha101_1'

    def calc(self, code_list, start_date, end_date) -> pd.DataFrame:
        """

        the example is just a day datasource, u can use the min data to generate a day-frequence factor

        the factor should be in day frequence 
        """
        data = QA.QA_fetch_stock_day_adv(code_list, start_date, end_date)

        cur_ret = smpl.get_current_return(data,'close')
        prices = data.data['close']

        factor = pd.DataFrame(alpha1(prices, cur_ret))
        factor = factor.replace([-np.inf, np.inf], 0).fillna(value=0)
        factor.columns = ['factor']

        return factor.reset_index().dropna()


In [6]:
from qaenv import (clickhouse_ip, clickhouse_password, clickhouse_port,
                   clickhouse_user)

from base.Constants import trade_date_sse
gap = 90
end = '2011-07-21'
start = trade_date_sse[trade_date_sse.index(end)-gap]
stock_list  = qaadapter.QA_adapter_get_code_from_block('sw_l1', '国防军工')

# print(smpl.get_stock_name(stock_list), len(stock_list))


factor_base = alpha101_1("alpha101_1", host=clickhouse_ip, port=clickhouse_port, user=clickhouse_user, password=clickhouse_password)
factor_base.update_to_database(stock_list, start, end)



['alpha101_1', 'factormetadata']


In [7]:
factor_base.fetch_data()

alpha101_1
date       code              
2011-03-11 000519    0.000000
           000547    0.000000
           000561    0.000000
           000576    0.000000
           000733   -0.389591
...                       ...
2023-07-21 603261   -0.379350
           603267    0.398269
           603678    0.208281
           603712    0.031412
           605123   -0.158040

[10896 rows x 1 columns]

In [1]:
from clickhouse_driver import Client
import clickhouse_driver

import QUANTAXIS as QA
import pandas as pd
port = 9001
host = 'localhost'
user = 'admin'
password = 'admin'
database = 'test'

#  database=database
client = clickhouse_driver.Client(host=host, port=port, user=user, database=database, password=password,
                                               settings={
                                                   'insert_block_size': 100000000},
                                               compression=True)
# 创建 ClickHouse 客户端连接

client.execute('CREATE DATABASE IF NOT EXISTS test')

# 创建表
client.execute('CREATE TABLE IF NOT EXISTS test_table (id Int, name String) ENGINE = MergeTree() ORDER BY id')

# 插入数据
client.execute('INSERT INTO test_table (id, name) VALUES', [(1, 'John'), (2, 'Alice'), (3, 'Bob')])

# 执行查询并获取结果
result = client.execute('SELECT * FROM test_table')

# 打印查询结果
for row in result:
    print(row)

# 更新数据
# client.execute('ALTER TABLE test_table UPDATE name = "HELLO" WHERE id = 2')
client.execute('ALTER TABLE test_table UPDATE name = \'HELLO\' WHERE id = 2')


# change = 'ALTER TABLE test_table MODIFY COLUMN id Int32;\
# \
# DELETE FROM test_table\
# WHERE id IN (\
#     SELECT id\
#     FROM (\
#         SELECT id, ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS row_num\
#         FROM test_table\
#     ) AS subquery\
#     WHERE row_num > 1\
# );'

# client.execute(change)

change = 'ALTER TABLE test_table MODIFY COLUMN id Int32;\
\
DELETE FROM test_table\
WHERE id IN (\
    SELECT id\
    FROM (\
        SELECT id, ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS row_num\
        FROM test_table\
    ) AS subquery\
    WHERE row_num > 1\
);'

client.execute(change)

# # client.execute("INSERT INTO test_table (id, name) VALUES (2, 'Mike')" )
result = client.execute('SELECT * FROM test_table')

# 打印查询结果
for row in result:
    print(row)
# # 删除数据
# client.execute('DELETE FROM test_table WHERE id = 3')

# 删除表
# client.execute('DROP TABLE IF EXISTS test_table')

jqdatasdk not installed
you are using non-interactive mdoel quantaxis
(1, 'John')
(2, 'Alice')
(3, 'Bob')
(1, 'John')
(1, 'John')
(1, 'John')
(1, 'John')
(2, 'HELLO')
(2, 'HELLO')
(2, 'HELLO')
(2, 'HELLO')
(3, 'Bob')
(3, 'Bob')
(3, 'Bob')
(3, 'Bob')


ServerException: Code: 62.
DB::Exception: Syntax error (Multi-statements are not allowed): failed at position 46 (end of query): ;DELETE FROM test_tableWHERE id IN (    SELECT id    FROM (        SELECT id, ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS row_num        FROM test_table  . . Stack trace:

0. /lib/x86_64-linux-gnu/libclickhouse.so.18.16(StackTrace::StackTrace()+0x16) [0x7f0c39755246]
1. /lib/x86_64-linux-gnu/libclickhouse.so.18.16(+0x3ce1af5) [0x7f0c3b9adaf5]
2. /lib/x86_64-linux-gnu/libclickhouse.so.18.16(+0x169417c) [0x7f0c3936017c]
3. /lib/x86_64-linux-gnu/libclickhouse.so.18.16(DB::parseQuery(DB::IParser&, char const*, char const*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, unsigned long)+0x2e) [0x7f0c3b4af53e]
4. /lib/x86_64-linux-gnu/libclickhouse.so.18.16(+0x3a7ef9d) [0x7f0c3b74af9d]
5. /lib/x86_64-linux-gnu/libclickhouse.so.18.16(DB::executeQuery(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<